In [1]:
import pymc_model as pm
import pymc
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import latin_hypercube as lh
import xarray as xr

This study is only focusing on glaciers ['03473'] in region [15].
(1, 882) (882,)
[ 0.] (1,)


/home/tushar/anaconda3/envs/pymc/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [4]:
s1 = pymc.database.pickle.load('testlength1000.pickle')
s3 = pymc.database.pickle.load('testlength3000.pickle')
s5 = pymc.database.pickle.load('testlength5000.pickle')
s10 = pymc.database.pickle.load('testlength10000.pickle')
s20 = pymc.database.pickle.load('testlength20000.pickle')
s30 = pymc.database.pickle.load('testlength30000.pickle')
sample_dict = {'1000':s1, '3000':s3, '5000':s5, '10000':s10, '20000':s20, '30000':s30}

In [5]:
dict_of_df = {}

for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = lh.sample2(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    print(type(result))
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)
    dict_of_df[key] = result

<class 'pandas.core.frame.DataFrame'>
1000 samples, mean: -0.5224335723606359 std: 0.22615401397843374
<class 'pandas.core.frame.DataFrame'>
3000 samples, mean: -0.5336070961171993 std: 0.22867745224315353
<class 'pandas.core.frame.DataFrame'>
5000 samples, mean: -0.5400767603988501 std: 0.24496480514985167
<class 'pandas.core.frame.DataFrame'>
10000 samples, mean: -0.5422913846983877 std: 0.23935924583026014
<class 'pandas.core.frame.DataFrame'>
20000 samples, mean: -0.5374301029385254 std: 0.23516596872796122
<class 'pandas.core.frame.DataFrame'>
30000 samples, mean: -0.5371995010776758 std: 0.23711770556341752


In [6]:
dict_of_df

{'1000':      tempchange  precfactor   ddfsnow   massbal  sorted_index
 0     -0.161266    0.838956  0.005501 -0.848848            72
 2     -0.378391    0.838956  0.004797 -0.682502           246
 4     -0.715481    0.838956  0.001886 -0.124318           963
 4     -0.715481    0.838956  0.001886 -0.124318           963
 5     -0.581967    0.777595  0.001886 -0.282471           861
 7     -0.404280    0.777595  0.001886 -0.374717           742
 9     -0.304408    0.777595  0.001886 -0.427821           658
 18    -0.584213    0.843783  0.003877 -0.495719           541
 21    -0.685341    0.769569  0.002453 -0.364162           753
 25    -0.592373    0.862222  0.003314 -0.412416           689
 28    -1.015629    0.862222  0.004414 -0.292579           849
 30    -0.900287    0.776112  0.004414 -0.454072           615
 32    -0.919968    0.599171  0.004354 -0.652973           286
 33    -0.888020    0.599171  0.004354 -0.668393           262
 37    -0.779870    0.515655  0.003456 -0.76565

Experiment with creating a DataArray

In [7]:
try1 = xr.DataArray(dict_of_df['1000'])

In [8]:
try1

<xarray.DataArray (dim_0: 300, dim_1: 5)>
array([[ -1.612661e-01,   8.389555e-01,   5.500872e-03,  -8.488482e-01,
          7.200000e+01],
       [ -3.783909e-01,   8.389555e-01,   4.797390e-03,  -6.825020e-01,
          2.460000e+02],
       [ -7.154808e-01,   8.389555e-01,   1.886115e-03,  -1.243180e-01,
          9.630000e+02],
       ..., 
       [  3.475907e-01,   1.336333e+00,   2.539863e-03,  -2.547346e-01,
          8.830000e+02],
       [  3.131501e-01,   1.336333e+00,   6.050470e-03,  -6.818438e-01,
          2.470000e+02],
       [  3.909058e-01,   1.336333e+00,   6.050470e-03,  -7.367852e-01,
          1.780000e+02]])
Coordinates:
  * dim_0    (dim_0) int64 0 2 4 4 5 7 9 18 21 25 28 30 32 33 37 43 50 51 52 ...
  * dim_1    (dim_1) object 'tempchange' 'precfactor' 'ddfsnow' 'massbal' ...